In [ ]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig

from gammapy.modeling.models import Models, SPECTRAL_MODEL_REGISTRY, SPATIAL_MODEL_REGISTRY
from gammapy.maps import Map

import matplotlib.pyplot as plt
import xmltodict
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import logging

In [ ]:
from asgardpy.data.dataset_3d import Dataset3DGeneration
from asgardpy.data.target import (
    set_models, 
    apply_selection_mask_to_models,
    create_gal_diffuse_skymodel, 
    create_source_skymodel, 
    read_models_from_asgardpy_config,
    xml_spectral_model_to_gammapy_params,
    xml_spatial_model_to_gammapy,
)

In [ ]:
log = logging.getLogger("test Models from XML")

In [ ]:
config_file = "/path/to/Config.yaml"

In [ ]:
config = AsgardpyConfig()

In [ ]:
%%time
config_main = config.read(config_file)

In [ ]:
for g in config_main.general:
    print(g)

# Steps mentioned in the main config file

In [ ]:
config_main.general.steps

# Target source information

In [ ]:
for c in config_main.target:
    print(c)

# 3D Datasets informations

In [ ]:
for cc in config_main.dataset3d:
    print(cc)

In [ ]:
%%time
analysis = AsgardpyAnalysis(config_main)

In [ ]:
analysis

# Get XML file from a single 3D Dataset - Fermi-LAT

In [ ]:
instruments_list = config_main.dataset3d.instruments
print(len(instruments_list), "number of 3D dataset information provided")

config_3d_dataset = instruments_list[0]
print(f"Instrument selected is {config_3d_dataset.name}")

key_names = config_3d_dataset.dataset_info.key
print(f"The list of different keys or modes of observations for the selected instrument are {key_names}")

In [ ]:
%%time
generate_3d_dataset = Dataset3DGeneration(
    log, config_3d_dataset, config_main
)

In [ ]:
if len(key_names) == 0:
    key_names = [None]

In [ ]:
%%time
file_list = generate_3d_dataset.read_to_objects(key_names[0])

In [ ]:
print(f"The XML file selected is {file_list['xml_file']}")
xml_file = file_list['xml_file']

# Read into Models from XML file

In [ ]:
with open(xml_file) as f:
    data = xmltodict.parse(f.read())["source_library"]["source"]

In [ ]:
print("The sources for which model information is given in the XML file, along with their spectral and spatial model:")
for source in data:
    print(source["@name"], source["spectrum"]["@type"], source["spatialModel"]["@type"])

In [ ]:
print("Diffuse Isotropic Model already initialized")
print(generate_3d_dataset.diffuse_models["iso_diffuse"])

In [ ]:
diff_gal = create_gal_diffuse_skymodel(generate_3d_dataset.diffuse_models["gal_diffuse"])
print(diff_gal)

In [ ]:
aux_path = generate_3d_dataset.config_3d_dataset.io[1].input_dir
print(aux_path)

In [ ]:
# Check if the Spectral Model type is properly translated
for source in data:
    source_name = source["@name"]
    if source_name not in ["IsoDiffModel", "GalDiffModel", "isodiff", "galdiff"]:
        spectrum_ = source["spectrum"]["parameter"]
        spectrum_type = source["spectrum"]["@type"].split("EblAtten::")[-1]

        if spectrum_[0]["@name"] not in ["GalDiffModel", "IsoDiffModel"]:
            if spectrum_type in ["PLSuperExpCutoff", "PLSuperExpCutoff2"]:
                spectrum_type_final = "ExpCutoffPowerLawSpectralModel"
            elif spectrum_type == "PLSuperExpCutoff4":
                spectrum_type_final = "SuperExpCutoffPowerLaw4FGLDR3SpectralModel"
            else:
                spectrum_type_final = f"{spectrum_type}SpectralModel"

            spectral_model = SPECTRAL_MODEL_REGISTRY.get_cls(spectrum_type_final)()

            if spectrum_type == "LogParabola":
                if "EblAtten" in source["spectrum"]["@type"]:
                    spectral_model = SPECTRAL_MODEL_REGISTRY.get_cls("PowerLawSpectralModel")()
                    ebl_atten_pl = True
                else:
                    spectral_model = SPECTRAL_MODEL_REGISTRY.get_cls(
                        "LogParabolaSpectralModel"
                    )()
        print(spectrum_type, spectral_model.tag)

In [ ]:
# Check if the Spectral Model parameters are properly translated
for source in data:
    source_name = source["@name"]

    if source_name not in ["IsoDiffModel", "GalDiffModel", "isodiff", "galdiff"]:
        print(f"\n{source_name}\n")
        spectrum_ = source["spectrum"]["parameter"]
        spectrum_type = source["spectrum"]["@type"].split("EblAtten::")[-1]

        print(spectrum_type)
        for p in spectrum_:
            print(f'Name: {p["@name"]:12s} \tScale: {float(p["@scale"]):6e} \tFrozen: {p["@free"]}')
            print(
                f'Value: {(float(p["@value"]) * float(p["@scale"])):.6e}' + 
                f' \tMin: {(float(p["@min"]) * float(p["@scale"])):.6e}' + 
                f' \tMax: {(float(p["@max"]) * float(p["@scale"])):.6e}'
            )

        if spectrum_[0]["@name"] not in ["GalDiffModel", "IsoDiffModel"]:
            if spectrum_type in ["PLSuperExpCutoff", "PLSuperExpCutoff2"]:
                spectrum_type_final = "ExpCutoffPowerLawSpectralModel"
            elif spectrum_type == "PLSuperExpCutoff4":
                spectrum_type_final = "SuperExpCutoffPowerLaw4FGLDR3SpectralModel"
            else:
                spectrum_type_final = f"{spectrum_type}SpectralModel"

            spectral_model = SPECTRAL_MODEL_REGISTRY.get_cls(spectrum_type_final)()

            if spectrum_type == "LogParabola":
                if "EblAtten" in source["spectrum"]["@type"]:
                    spectral_model = SPECTRAL_MODEL_REGISTRY.get_cls("PowerLawSpectralModel")()
                    ebl_atten_pl = True
                else:
                    spectral_model = SPECTRAL_MODEL_REGISTRY.get_cls(
                        "LogParabolaSpectralModel"
                    )()
        # Read the parameter values from XML file to create SpectralModel
        params_list = xml_spectral_model_to_gammapy_params(
            spectrum_,
            spectrum_type,
        )
        print("\n", spectral_model.tag)
        for p in params_list:
            print(f"Name: {p.name:12s} \tScale: {p.scale} \t\tFrozen:{p.frozen}")
            print(f"Value: {p.value:.6e} \tMin: {p.min:.6e} \tMax: {p.max:.6e}")


# Get the list of source models

In [ ]:
list_sources = []
is_target_source = False

for source in data:
    source_name = source["@name"]
    if source_name not in ["IsoDiffModel", "GalDiffModel", "isodiff", "galdiff"]:
        source, is_target_source = create_source_skymodel(
            config_main.target, source, aux_path
        )

        if is_target_source:
            list_sources.insert(0, source)
        else:
            list_sources.append(source)

In [ ]:
list_sources.append(generate_3d_dataset.diffuse_models["iso_diffuse"])

In [ ]:
print(len(list_sources))

In [ ]:
for m in list_sources:
    print(m)

In [ ]:
target_pos = Models(list_sources)[config_main.target.source_name].spatial_model.position.icrs
for m in list_sources:
    print("\n", m.name)
    if m.spatial_model.position:
        print(f"Separation from Target source is {target_pos.separation(m.spatial_model.position.icrs).deg:.3f} deg")
    if m.spectral_model.tag[1] == "compound":
        print(m.spectral_model.model1.tag[0])
    else:
        print(m.spectral_model.tag[0])
    for p in m.spectral_model.parameters:
        print(p.name, p._is_norm)

In [ ]:
plt.figure(figsize=(10,5))
en_b = [1e-4, 1e2] * u.TeV
for m in list_sources:
    if m.spatial_model.position:
        label_ = f"{m.name} {target_pos.separation(m.spatial_model.position.icrs).deg:.1e} deg"
    else:
        label_ = m.name
    m.spectral_model.plot(energy_bounds=en_b, sed_type="e2dnde", label=label_)

plt.grid()
plt.legend(bbox_to_anchor=(1,1), ncols=int(np.sqrt(len(list_sources))/2))
plt.ylim(1e-13, 1e-8)

# Spatial Models

In [ ]:
for source in data:
    print(source["@name"])
    xml_spatial_model = source["spatialModel"]
    spatial_pars = xml_spatial_model["parameter"]

    if xml_spatial_model["@type"] == "SkyDirFunction":
        for par_ in spatial_pars:
            print(par_)
            if par_["@name"] == "RA":
                lon_0 = f"{par_['@value']} deg"
            if par_["@name"] == "DEC":
                lat_0 = f"{par_['@value']} deg"
        fk5_frame = SkyCoord(
            lon_0,
            lat_0,
            frame="fk5",
        )
        gal_frame = fk5_frame.transform_to("galactic")
        spatial_model = SPATIAL_MODEL_REGISTRY.get_cls("PointSpatialModel")().from_position(
            gal_frame
        )

    elif xml_spatial_model["@type"] == "SpatialMap":
        file_name = xml_spatial_model["@file"].split("/")[-1]
        file_path = aux_path / f"Templates/{file_name}"

        spatial_map = Map.read(file_path)
        spatial_map = spatial_map.copy(unit="sr^-1")

        spatial_model = SPATIAL_MODEL_REGISTRY.get_cls("TemplateSpatialModel")(
            spatial_map, filename=file_path
        )

    elif xml_spatial_model["@type"] == "RadialGaussian":
        for par_ in spatial_pars:
            print(par_)
            if par_["@name"] == "RA":
                lon_0 = f"{par_['@value']} deg"
            if par_["@name"] == "DEC":
                lat_0 = f"{par_['@value']} deg"
            if par_["@name"] == "Sigma":
                sigma = f"{par_['@value']} deg"
        fk5_frame_ = SkyCoord(
            lon_0,
            lat_0,
            frame="fk5",
        )
        print(fk5_frame_)
        gal_frame = fk5_frame_.transform_to("galactic")

        spatial_model = SPATIAL_MODEL_REGISTRY.get_cls("GaussianSpatialModel")(
            lon_0=gal_frame.l, lat_0=gal_frame.b, sigma=sigma, frame="galactic"
        )

    print(spatial_model.position)


In [ ]:
for m in list_sources:
    print("\n", m.name, m.spatial_model.tag)
    print(m.spatial_model, m.spatial_model.evaluation_radius)

In [ ]:
idx = 0
print(list_sources[idx].name, list_sources[idx].spatial_model)
test_ = list_sources[idx].spatial_model
test_.plot(add_cbar=True)

# Reading models from Config file

In [ ]:
spectral_model, spatial_model = read_models_from_asgardpy_config(config_main.target)

In [ ]:
print(spectral_model)

In [ ]:
print(spatial_model)

# Apply selection mask to list of models

In [ ]:
list_sources = apply_selection_mask_to_models(
    list_sources,
    target_source=config_main.target.source_name,
    roi_radius=config_main.target.roi_selection.roi_radius,
    free_sources=config_main.target.roi_selection.free_sources
)

In [ ]:
for m in list_sources:
    print(m)

In [ ]:
print(len(list_sources))

In [ ]:
print(len(list_sources.parameters.free_parameters))